# InSituPy demonstration - Add annotations

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy import read_xenium

## Previous steps

1. Download the example data for demonstration: [01_InSituPy_demo_download_data.ipynb](./01_InSituPy_demo_download_data.ipynb)
2. Register images from external stainings: [02_InSituPy_demo_register_images.ipynb](./02_InSituPy_demo_register_images.ipynb)
3. Visualize data with napari and do preprocessing steps: [03_InSituPy_demo_analyze.ipynb](./03_InSituPy_demo_analyze.ipynb)

At this point, the structure of the data should look like this:

    ```
    ./demo_dataset
    ├───cropped_processed
    ├───output-XETG00000__slide_id__sample_id
    │   ├───analysis
    │   │   ├───clustering
    │   │   ├───diffexp
    │   │   ├───pca
    │   │   ├───tsne
    │   │   └───umap
    │   └───cell_feature_matrix
    ├───registered_images
    ├───registration_qc
    └───unregistered_images
    ```


## Load Xenium data into `XeniumData` object

Now the Xenium data can be parsed by providing the data path to `XeniumData`

In [3]:
insitupy_project = Path("demo_dataset/demo_insitupy_project")

In [4]:
xd = read_xenium(insitupy_project)

In [5]:
xd

XeniumData
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	demo_dataset
Data folder:	demo_insitupy_project
Metadata file:	.ispy

In [6]:
xd.load_images()
xd.load_cells()

Loading images...
Loading cells...
	No alternative cells found...


Note: That the `annotations` and `regions` modalities are not found here is expected. Annotations and regions are added in a later step.

In [7]:
xd

XeniumData
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	demo_dataset
Data folder:	demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear

## Load annotations

For the analysis of spatial transcriptomic datasets the inclusion of annotations from experts of disease pathology is key. Here, we demonstrate how to annotate data in [QuPath](https://qupath.github.io/), export the annotations as `.geojson` file and import them into the `XeniumData` object.

### Create annotations in QuPath

To create annotations in QuPath, follow these steps:

1. Select a annotation tool from the bar on the top left:

<center><img src="./demo_annotations/qupath_annotation_buttons.jpg" width="300"/></center>

2. Add as many annotations as you want and label them by setting classes in the annotation list. Do not forget to press the "Set class" button:

<center><img src="./demo_annotations/qupath_annotation_list.jpg" width="350"/></center>

3. Export annotations using `File > Export objects as GeoJSON`. Tick `Pretty JSON` to get an easily readable JSON file. The file name needs to have following structure: `annotation-{slide_id}__{sample_id}__{annotation_label}`.

### Import annotations into `XeniumData`

For demonstration purposes, we created a dummy annotation file in `./demo_annotations/`. To add the annotations to `XeniumData` follow the steps below.



In [8]:
xd.import_annotations(
    files=[
        "./demo_annotations/annotations-0001879__Replicate 1__demo.geojson",
        "./demo_annotations/annotations-0001879__Replicate 1__demo2.geojson"
           ],
    keys=["demo", "demo2"]
    )

Importing annotations...


C:\Users\ge37voy\Github\InSituPy\insitupy\_core\dataclasses.py:66: UserWarning: Some annotations were a shapely 'MultiPolygon' objects and skipped.
  self.add_shapes(data=file,


## Load regions

Regions can be created in QuPath either as described above or using tools like the TMA dearrayer. They are also exported as objects as annotations but different to annotations they do not have a classification and each name of a region has to be unique.

In the following demo regions are read. One of the region files has non-unique names to demonstrate the warning that appears in this case.

In [10]:
xd.import_regions(
    files=[
        "./demo_regions/regions-0001879__Replicate 1__demo_regions.geojson",
        "./demo_regions/regions-0001879__Replicate 1__TMA.geojson",
        ],
    keys=['demo_regions', 'TMA'])

Importing regions...


Properties of the added `anotations` and `regions` can be inspected in the XeniumData representation:

In [11]:
xd

XeniumData
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	demo_dataset
Data folder:	demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	4 annotations, 3 classes ('Negative','Positive','Other') 
    ➤ regions
       demo_regions:	3 regions, 3 classes ('Region1','Region2','Region3') 
       TMA:	6 regions, 6 classes ('B-2','A-3','

Both annotations and regions are saved as [`GeoPandas`](https://geopandas.org/en/stable/) dataframes. The main difference between both is that `regions` need to have a unique name while `annotations` are allowed to have duplicate names. In both cases a unique `id` is used for identification.

In [12]:
xd.annotations.demo2

,objectType,geometry,name,color,origin
id,,,,,
1970eccb-ad38-4b4b-b7a8-54509027b57d,annotation,"POLYGON ((5380.28750 827.05000, 5379.01250 827...",Negative,"[112, 112, 225]",file
a3b32cce-1bb9-4a6f-b1d1-9e0c44420cfa,annotation,"POLYGON ((6576.87500 2306.68750, 6575.60000 23...",Positive,"[250, 62, 62]",file
a6c17a54-6839-40b2-8531-c9227635f344,annotation,"POLYGON ((1381.46250 3639.27500, 1380.18750 36...",Other,"[255, 200, 0]",file
e78efe2f-d185-4ab6-9cc9-6621897f3662,annotation,"POLYGON ((6272.92137 3936.13750, 6263.65000 39...",Negative,"[112, 112, 225]",file


In [13]:
xd.regions.TMA

,objectType,name,isMissing,geometry,origin
id,,,,,
7ab5d5a6-49bd-4122-bc64-05477bc0207b,tmaCore,B-2,False,"POLYGON ((4299.61025 4213.18862, 4298.62425 42...",file
06ef93c1-f86d-45e6-ad9a-896e254638ea,tmaCore,A-3,False,"POLYGON ((7201.19150 903.26738, 7200.20550 934...",file
7933d3fd-ccd3-46af-8f15-fcc01ec9c128,tmaCore,B-1,False,"POLYGON ((1555.14725 4333.64638, 1554.15912 43...",file
7015118d-2947-48e3-baf0-4b220a76a951,tmaCore,B-3,False,"POLYGON ((7311.17300 4228.90087, 7310.18700 42...",file
bf86657f-31f6-40fe-983b-f80c3d75512b,tmaCore,A-1,False,"POLYGON ((1481.82625 908.50338, 1480.83812 939...",file
440d8f00-fb0e-42e7-9f98-30d30adfc8df,tmaCore,A-2,False,"POLYGON ((4173.91650 856.13063, 4172.93050 887...",file


### Visualize and edit annotations and regions using napari

To visualize annotations and regions in napari, three widgets are available:

<left><img src="./demo_annotations/napari_region+annotations_widget.jpg" width="300"/></left>

Using this widget, annotations and regions can be displayed. Currently only annotations can be added using the respective widget.

Alternatively, annotations can be displayed while starting the napari with `.show()` using the `annotation_keys` argument:


In [14]:
xd.show(annotation_keys="all")

c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is tr

#### Annotation layers

The annotations are added as shapes layers to the layer list. The layer name always starts with a "*" and has following syntax: `"* Class (Label)"`:

<left><img src="./demo_annotations/napari_layerlist_annotations.jpg" width="300"/></left>

- **Label**: A label for one collection of annotations. Could e.g. tell us who did the annotations or what is the focus of this collection of annotations.
- **Class**: Specifies the class of one specific annotation. Could be e.g. the name of cells, the morphological structure or the disease state annotated.

#### Add custom annotations using the Annotation Widget

<left><img src="./demo_annotations/napari_annotation_widget.jpg" width="200"/></left>

By clicking the `"Add annotation layer"` button a new layer with the above mentioned syntax is added. The layer controls on the top left can be then used to add new shapes as annotations:

<left><img src="./demo_annotations/napari_layerconrols_annotations.jpg" width="300"/></left>

An example annotation is shown here:

<left><img src="./demo_annotations/napari_annotation_example.jpg" width="200"/></left>

The annotations can then be stored in the `XeniumData` object using the `store_annotations` function.


In [15]:
xd.store_annotations()

Added 3 new annotations to existing key 'demo2'


In [16]:
xd

XeniumData
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	demo_dataset
Data folder:	demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	7 annotations, 4 classes ('Negative','Positive','Other','test_class') 
    ➤ regions
       demo_regions:	3 regions, 3 classes ('Region1','Region2','Region3') 
       TMA:	6 regions, 6 classes (

### Assign annotations to observations

To use the annotations in analyses (e.g. to select only observations within a certain annotation or compare gene expression between different annotations) one can use the `assign_annotations` function. It adds columns containing the annotation class to `xd.matrix.obs`. The column has the syntax `annotation-{Label}` and if an observation is not part of any annotation within this label, it contains `NaN`. 

In [17]:
xd.assign_annotations()

Assigning key 'demo'...
Assigning key 'demo2'...


After assigning the annotations, the labels analyzed here are marked with a ✔:

In [18]:
xd

XeniumData
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	demo_dataset
Data folder:	demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') ✔
       demo2:	7 annotations, 4 classes ('Negative','Positive','Other','test_class') ✔
    ➤ regions
       demo_regions:	3 regions, 3 classes ('Region1','Region2','Regi

Following cells show examples how to explore the assigned annotations:

In [19]:
# print number of cells within one annotation
xd.cells.matrix.obs["annotation-demo2"].notna().sum()

21199

In [20]:
# show only observations that were part of this annotation label
xd.cells.matrix.obs[xd.cells.matrix.obs["annotation-demo2"].notna()]

,transcript_counts,control_probe_counts,control_codeword_counts,total_counts,cell_area,nucleus_area,annotation-demo,annotation-demo2
4921,281,0,0,281,733.247187,26.010000,NaN,Other
4922,273,1,0,274,380.576875,30.074063,NaN,Other
4923,189,2,0,191,285.658437,8.263594,NaN,Other
4924,212,0,0,212,282.226562,24.068281,NaN,Other
4925,58,0,0,58,81.823125,4.470469,NaN,Other
...,...,...,...,...,...,...,...,...
165374,96,1,0,97,150.234844,11.063281,Negative,Negative
165375,379,0,0,379,153.666719,75.681875,Negative,Negative
165376,101,0,0,101,27.996875,17.836719,Negative,Negative
165377,472,0,0,472,200.177656,52.652188,Negative,Negative


## Save imported annotations in `InSituPy` project

In [25]:
xd.save()

Updating project in c:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
	Updating cells...
	Updating annotations...
	Updating regions...
Saved.
Reloading following modalities: annotations,cells,images,regions
Loading annotations...
Loading cells...
	No alternative cells found...
Loading images...
Loading regions...


In [26]:
xd.show()